energy consumption vs unemployment: https://www.kaggle.com/jayrav13/unemployment-by-county-us
energy consumption vs storms / weather

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 90)
import warnings
warnings.filterwarnings('ignore')

In [2]:
""" 
AEP = pd.read_csv("AEP_hourly.csv", memory_map=True)
COMED = pd.read_csv("COMED_hourly.csv", memory_map=True)
DAYTON = pd.read_csv("DAYTON_hourly.csv", memory_map=True)
DEOK = pd.read_csv("DEOK_hourly.csv", memory_map=True)
DOM = pd.read_csv("DOM_hourly.csv", memory_map=True)
DUQ = pd.read_csv("DUQ_hourly.csv", memory_map=True)
EKPC = pd.read_csv("EKPC_hourly.csv", memory_map=True)
FE = pd.read_csv("FE_hourly.csv", memory_map=True)
NI = pd.read_csv("NI_hourly.csv", memory_map=True)
PJM_est = pd.read_csv("pjm_hourly_est.csv", memory_map=True)
PJM_Load = pd.read_csv("PJM_Load_hourly.csv", memory_map=True)
PJME = pd.read_csv("PJME_hourly.csv", memory_map=True)
PJMW = pd.read_csv("PJMW_hourly.csv", memory_map=True)
"""

energy_gdp_by_state_10_14 = pd.read_csv("Census_and_economic_2010_2014.csv", memory_map=True) 
unemployment_90_16 = pd.read_csv("unemployment_by_county.csv", memory_map=True)
gdp_per_cap_92_16 = pd.read_csv("gdp_per_capita.csv", memory_map=True)
gdp_per_cap_92_16.drop('Unnamed: 0', axis=1, inplace=True) 

In [3]:
# Get rid of all years not in our census dataset
unemp_10_14 = unemployment_90_16[(unemployment_90_16['Year']>=2010) & (unemployment_90_16['Year']<=2014)]

# Set index to index column
unemp_10_14.set_index(unemp_10_14.index)

# Remove columns that won't be used
unemp_10_14_short = unemp_10_14.drop(['Month', 'County'], axis=1)

# Set index to start at 0 again
unemp_10_14_short.set_index(np.arange(0,len(unemp_10_14_short), 1), inplace=True)

# Calculate the average unemployment rate for each state, by year. This removes all excess rows to keep just year, state and rate
unemp_10_14_avg = pd.Series.to_frame(unemp_10_14_short.groupby(['Year','State'])['Rate'].mean())
unemp_10_14_avg.reset_index(inplace=True)
unemp_10_14_avg.head()

# Shorten even farther to have 1 row per state. Unemployment rate will then have 1 column for each year. 
# This format is to match the census file

unemp_avg_by_state = unemp_10_14_avg.pivot(index='State', columns='Year', values='Rate')
unemp_avg_by_state.reset_index(inplace=True)
unemp_avg_by_state.columns = ['State', 'Unemp_rate2010', 'Unemp_rate2011', 'Unemp_rate2012', 'Unemp_rate2013', 'Unemp_rate2014']


In [4]:
#Remove excess years not in the main table
gdp_per_cap_10_14 = gdp_per_cap_92_16[(gdp_per_cap_92_16['YEAR'] >= 2010)& (gdp_per_cap_92_16['YEAR'] <= 2014)]
gdp_per_cap_10_14.set_index(np.arange(0,len(gdp_per_cap_10_14),1), inplace=True)

#Pivot table to have 1 row per state like in the main table
gdp_by_state = gdp_per_cap_10_14.pivot(index='STATE', columns='YEAR', values='gdp_per_capita')
gdp_by_state.reset_index(inplace=True)
gdp_by_state.columns = ['State', 'GDP_per_cap2010', 'GDP_per_cap2011', 'GDP_per_cap2012', 'GDP_per_cap2013', 'GDP_per_cap2014']

The main table for this project contains 52 "states", 50 are actual states, and it includes 2 others:

* District of Columbia
* United States

By look at length and unique values, it was found that the unemployment table is missing 5 values from the main table:

* Alaska 
* Florida 
* Georgia 
* District of Columbia
* United States

By looking at the unique values it was found that the GDP table has 8 extra values from the main table:

* Far West
* Great Lakes
* Mideast
* New England
* Plains
* Rocky Mountain
* Southeast
* Southwest

Next step will be to clean up the main census file that contains 192 columns and 50 rows. Any missing data will be found, and correlations between different columns will begin to be explored.

* Can the table to transformed (pivot_table) so that each year is on its own row?
* If this is the way that will be used the pivot transformations of the 2 other tables will be removed
* The 3 tables then need to be merged and explored


In [5]:
first_2 = pd.merge(energy_gdp_by_state_10_14, gdp_by_state, left_on='State', right_on='State', how='left')
df = pd.merge(first_2, unemp_avg_by_state, left_on='State', right_on='State', how='left')
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52 entries, 0 to 51
Columns: 202 entries, StateCodes to Unemp_rate2014
dtypes: float64(104), int64(96), object(2)
memory usage: 82.5+ KB


In [6]:
# Separate out columns of our dataframe by year so we can have each row represent a year
cols_2010 = [col for col in df.columns if '2010' in col]
cols_2011 = [col for col in df.columns if '2011' in col]
cols_2012 = [col for col in df.columns if '2012' in col]
cols_2013 = [col for col in df.columns if '2013' in col]
cols_2014 = [col for col in df.columns if '2014' in col]
cols_diff_yrs = [col for col in df.columns if '-' in col]
other_cols = list(set(df.columns) - set(cols_2010) - set(cols_2011) - set(cols_2012) - set(cols_2013) - set(cols_2014) - set(cols_diff_yrs))

In [7]:
# Get columns of dataframe from previous lists of column names
cols_other_2010 = list(other_cols + cols_2010)
cols_other_2011 = list(other_cols + cols_2011)
cols_other_2012 = list(other_cols + cols_2012)
cols_other_2013 = list(other_cols + cols_2013)
cols_other_2014 = list(other_cols + cols_2014)
df_2010 = df[cols_other_2010]
df_2011 = df[cols_other_2011]
df_2012 = df[cols_other_2012]
df_2013 = df[cols_other_2013]
df_2014 = df[cols_other_2014]
df_2010.head()

,Great Lakes,State,Division,Region,Coast,StateCodes,TotalC2010,TotalP2010,TotalE2010,TotalPrice2010,BiomassC2010,CoalC2010,CoalP2010,CoalE2010,CoalPrice2010,ElecC2010,ElecE2010,ElecPrice2010,FossFuelC2010,GeoC2010,GeoP2010,HydroC2010,HydroP2010,NatGasC2010,NatGasE2010,NatGasPrice2010,LPGC2010,LPGE2010,LPGPrice2010,GDP2010Q1,GDP2010Q2,GDP2010Q3,GDP2010Q4,GDP2010,CENSUS2010POP,POPESTIMATE2010,GDP_per_cap2010,Unemp_rate2010
0,0.0,Alabama,6.0,3.0,1.0,AL,1931522,1419613,21630.5,17.82,169088,718684,493094,2135.6,2.97,310023,7833.0,26.44,1785688,138,0,84919,8704,544405,3222.2,6.65,13088,316.7,24.20,172033,175198,178520,179132,176220.75,4779736,4785822,35913,12.360697
1,0.0,Alaska,9.0,4.0,1.0,AK,653221,1738207,6473.9,20.13,4178,14548,33556,49.9,3.43,21315,912.4,43.29,634777,153,0,13982,1433,334978,479.2,6.41,1353,35.6,26.79,52307,53673,54846,56054,54220.00,710231,713856,69564,NaN
2,0.0,Arizona,8.0,4.0,0.0,AZ,1383531,580948,19372.7,22.25,29289,457909,167930,829.1,1.81,248506,7058.8,28.40,1292346,339,0,64606,6622,336208,2198.8,6.86,7838,219.9,28.05,242824,246487,248985,251036,247333.00,2915918,2922297,37935,13.060000
3,0.0,Arkansas,7.0,3.0,0.0,AR,1120632,1247709,12269.4,16.90,94865,293689,718,510.6,1.74,164439,3393.3,21.57,909827,773,0,35697,3659,274774,1791.7,7.23,10016,224.3,22.40,101063,104880,106687,107085,104928.75,6392017,6411999,34671,9.101111
4,0.0,California,9.0,4.0,1.0,CA,7760629,2532205,117090.0,20.97,280124,54972,0,161.4,2.94,882107,33381.8,38.23,5741492,124981,12600,326152,33431,2325411,14914.1,6.97,63022,1669.8,27.58,1920145,1943428,1963650,2016517,1960935.00,37253956,37336011,51871,13.893276


In [8]:
# Add Year column to each and drop year from individual column names
df_2010['Year'] = 2010
df_2011['Year'] = 2011
df_2012['Year'] = 2012
df_2013['Year'] = 2013
df_2014['Year'] = 2014

df_2010.columns = [col.replace('2010', '') for col in df_2010.columns]
df_2011.columns = [col.replace('2011', '') for col in df_2011.columns]
df_2012.columns = [col.replace('2012', '') for col in df_2012.columns]
df_2013.columns = [col.replace('2013', '') for col in df_2013.columns]
df_2014.columns = [col.replace('2014', '') for col in df_2014.columns]


In [9]:
df_full = pd.concat([df_2010, df_2011, df_2012, df_2013, df_2014], ignore_index=True, sort=True)
df_full.head()

,BiomassC,CENSUSPOP,CoalC,CoalE,CoalP,CoalPrice,Coast,Division,ElecC,ElecE,ElecPrice,FossFuelC,GDP,GDPQ1,GDPQ2,GDPQ3,GDPQ4,GDP_per_cap,GeoC,GeoP,Great Lakes,HydroC,HydroP,LPGC,LPGE,LPGPrice,NatGasC,NatGasE,NatGasPrice,POPESTIMATE,RBIRTH,RDEATH,RDOMESTICMIG,RINTERNATIONALMIG,RNATURALINC,RNETMIG,Region,State,StateCodes,TotalC,TotalE,TotalP,TotalPrice,Unemp_rate,Year
0,169088,4779736.0,718684,2135.6,493094,2.97,1.0,6.0,310023,7833.0,26.44,1785688,176220.75,172033,175198,178520,179132,35913,138,0,0.0,84919,8704,13088,316.7,24.20,544405,3222.2,6.65,4785822,NaN,NaN,NaN,NaN,NaN,NaN,3.0,Alabama,AL,1931522,21630.5,1419613,17.82,12.360697,2010
1,4178,710231.0,14548,49.9,33556,3.43,1.0,9.0,21315,912.4,43.29,634777,54220.00,52307,53673,54846,56054,69564,153,0,0.0,13982,1433,1353,35.6,26.79,334978,479.2,6.41,713856,NaN,NaN,NaN,NaN,NaN,NaN,4.0,Alaska,AK,653221,6473.9,1738207,20.13,NaN,2010
2,29289,2915918.0,457909,829.1,167930,1.81,0.0,8.0,248506,7058.8,28.40,1292346,247333.00,242824,246487,248985,251036,37935,339,0,0.0,64606,6622,7838,219.9,28.05,336208,2198.8,6.86,2922297,NaN,NaN,NaN,NaN,NaN,NaN,4.0,Arizona,AZ,1383531,19372.7,580948,22.25,13.060000,2010
3,94865,6392017.0,293689,510.6,718,1.74,0.0,7.0,164439,3393.3,21.57,909827,104928.75,101063,104880,106687,107085,34671,773,0,0.0,35697,3659,10016,224.3,22.40,274774,1791.7,7.23,6411999,NaN,NaN,NaN,NaN,NaN,NaN,3.0,Arkansas,AR,1120632,12269.4,1247709,16.90,9.101111,2010
4,280124,37253956.0,54972,161.4,0,2.94,1.0,9.0,882107,33381.8,38.23,5741492,1960935.00,1920145,1943428,1963650,2016517,51871,124981,12600,0.0,326152,33431,63022,1669.8,27.58,2325411,14914.1,6.97,37336011,NaN,NaN,NaN,NaN,NaN,NaN,4.0,California,CA,7760629,117090.0,2532205,20.97,13.893276,2010


In [10]:
df_full.describe()

,BiomassC,CENSUSPOP,CoalC,CoalE,CoalP,CoalPrice,Coast,Division,ElecC,ElecE,ElecPrice,FossFuelC,GDP,GDPQ1,GDPQ2,GDPQ3,GDPQ4,GDP_per_cap,GeoC,GeoP,Great Lakes,HydroC,HydroP,LPGC,LPGE,LPGPrice,NatGasC,NatGasE,NatGasPrice,POPESTIMATE,RBIRTH,RDEATH,RDOMESTICMIG,RINTERNATIONALMIG,RNATURALINC,RNETMIG,Region,TotalC,TotalE,TotalP,TotalPrice,Unemp_rate,Year
count,2.600000e+02,5.200000e+01,2.600000e+02,260.000000,2.600000e+02,260.000000,255.00000,255.000000,2.600000e+02,260.000000,260.000000,2.600000e+02,2.600000e+02,2.600000e+02,2.600000e+02,2.600000e+02,2.600000e+02,260.000000,260.000000,260.000000,255.000000,2.600000e+02,260.000000,2.600000e+02,260.000000,260.000000,2.600000e+02,260.000000,260.000000,2.600000e+02,208.000000,208.000000,204.000000,208.000000,208.000000,208.000000,255.000000,2.600000e+02,2.600000e+02,2.600000e+02,260.000000,235.000000,260.000000
mean,1.680972e+05,1.187483e+07,7.223732e+05,1821.498846,8.042039e+05,2.633231,0.45098,5.117647,4.905118e+05,14282.506923,30.733654,3.060784e+06,6.160688e+05,6.058749e+05,6.131483e+05,6.196974e+05,6.255547e+05,49227.480769,8156.761538,598.080769,0.156863,1.023004e+05,10644.042308,1.140657e+05,2187.716923,26.405500,1.001217e+06,5966.972308,7.941769,1.208104e+07,12.602127,8.367475,0.403579,2.460574,4.234652,2.856392,2.666667,3.724819e+06,5.180297e+04,2.987990e+06,21.822923,7.834320,2012.000000
std,5.940948e+05,4.251612e+07,2.567657e+06,6472.319233,3.035151e+06,0.973320,0.49857,2.514893,1.736620e+06,50659.732465,11.530311,1.087910e+07,2.197262e+06,2.160755e+06,2.186693e+06,2.210344e+06,2.231306e+06,18277.358258,33205.554662,2720.783706,0.364386,3.811399e+05,39640.276827,4.577201e+05,8523.784320,4.444317,3.575514e+06,21269.451525,4.957073,4.292796e+07,1.517042,1.190846,4.358204,1.549112,2.359857,4.410284,1.043666,1.322080e+07,1.841678e+05,1.110214e+07,3.730914,2.260532,1.416941
min,8.190000e+02,5.636260e+05,0.000000e+00,0.000000,0.000000e+00,0.000000,0.00000,1.000000,1.880300e+04,740.800000,18.270000,4.902500e+04,2.640650e+04,2.571500e+04,2.632100e+04,2.672900e+04,2.686100e+04,31167.000000,6.000000,0.000000,0.000000,0.000000e+00,0.000000,1.800000e+01,0.500000,13.860000,2.732000e+03,91.800000,3.270000,5.643580e+05,9.313705,5.236680,-13.754494,0.557539,-0.776457,-10.884734,1.000000,1.304120e+05,2.218200e+03,9.500000e+01,13.790000,3.094713,2010.000000
25%,2.797425e+04,1.761651e+06,3.937975e+04,129.175000,0.000000e+00,1.990000,0.00000,3.000000,7.903100e+04,2088.025000,24.342500,6.061778e+05,8.375225e+04,8.244250e+04,8.267350e+04,8.378875e+04,8.610425e+04,40421.500000,337.000000,0.000000,0.000000,4.758500e+03,498.750000,7.701000e+03,190.350000,23.505000,1.617755e+05,742.125000,6.317500,1.781015e+06,11.921829,7.648860,-1.950719,1.365667,2.876569,0.002172,2.000000,6.484068e+05,8.019275e+03,2.486005e+05,19.495000,6.137025,2011.000000
50%,6.835850e+04,4.436370e+06,3.021140e+05,636.600000,1.940000e+02,2.585000,0.00000,5.000000,1.961375e+05,5499.350000,27.080000,1.113105e+06,1.942938e+05,1.918160e+05,1.938130e+05,1.947330e+05,1.936980e+05,46228.500000,958.000000,0.000000,0.000000,1.361950e+04,1427.500000,1.115350e+04,314.650000,26.250000,2.940025e+05,2004.950000,7.100000,4.479519e+06,12.550321,8.254358,-0.421704,1.901945,4.045486,1.709057,3.000000,1.440417e+06,1.988430e+04,6.583325e+05,21.345000,7.750453,2012.000000
75%,1.324628e+05,7.043661e+06,5.077322e+05,1523.300000,4.221975e+05,3.365000,1.00000,8.000000,3.518022e+05,9624.625000,33.837500,1.787826e+06,4.173401e+05,4.128920e+05,4.168898e+05,4.174525e+05,4.209722e+05,52029.500000,2042.000000,0.000000,0.000000,3.470050e+04,3632.000000,2.810275e+04,688.775000,29.517500,6.530788e+05,4011.625000,8.197500,7.302492e+06,13.364880,9.197824,3.176908,3.142864,5.445936,5.121678,4.000000,2.481635e+06,3.435455e+04,1.582822e+06,23.755000,9.138070,2013.000000
max,4.611638e+06,3.087455e+08,2.082838e+07,50473.700000,2.205725e+07,4.900000,1.00000,9.000000,1.284516e+07,389871.500000,99.960000,8.089132e+07,1.723314e+07,1.686974e+07,1.715519e+07,1.740699e+07,1.750

In [11]:
# df_full.info()

In [12]:
# Dropping CENSUSPOP column due to missing ~80% of the data
df_full.drop(['CENSUSPOP'], axis=1, inplace=True)

In [13]:
# Break up consumption, expenditure, and production columns
consumption_cols = [col for col in df_full.columns if col[-1] == 'C']
expenditure_cols = [col for col in df_full.columns if col[-1] == 'E']
production_cols = [col for col in df_full.columns if col[-1] == 'P']
other_cols = list(set(df_full.columns) - set(consumption_cols) - set(expenditure_cols) - set(production_cols))

print(consumption_cols)
print(expenditure_cols)
print(production_cols)
print(other_cols)

['BiomassC', 'CoalC', 'ElecC', 'FossFuelC', 'GeoC', 'HydroC', 'LPGC', 'NatGasC', 'RNATURALINC', 'TotalC']
['CoalE', 'ElecE', 'LPGE', 'NatGasE', 'POPESTIMATE', 'TotalE']
['CoalP', 'GDP', 'GeoP', 'HydroP', 'TotalP']
['Great Lakes', 'RINTERNATIONALMIG', 'GDPQ1', 'Year', 'State', 'GDPQ3', 'TotalPrice', 'LPGPrice', 'Unemp_rate', 'RDOMESTICMIG', 'Coast', 'StateCodes', 'ElecPrice', 'GDPQ2', 'GDP_per_cap', 'Division', 'NatGasPrice', 'CoalPrice', 'GDPQ4', 'RDEATH', 'RNETMIG', 'Region', 'RBIRTH']


In [14]:
# Get column names for each energy type 
energy_types = ['Biomass', 'Coal', 'Elec', 'Geo', 'Hydro', 'NatGas', 'LPG']
energy_lists = [[], [], [], [], [], [], []]
non_energy = df_full.columns
for idx, energy in enumerate(energy_types):
    energy_lists[idx] = [col for col in df_full.columns if energy in col]
    non_energy = list(set(non_energy) - set(energy_lists[idx]))
print(energy_lists)
print(non_energy)

[['BiomassC'], ['CoalC', 'CoalE', 'CoalP', 'CoalPrice'], ['ElecC', 'ElecE', 'ElecPrice'], ['GeoC', 'GeoP'], ['HydroC', 'HydroP'], ['NatGasC', 'NatGasE', 'NatGasPrice'], ['LPGC', 'LPGE', 'LPGPrice']]
['Great Lakes', 'RINTERNATIONALMIG', 'FossFuelC', 'TotalC', 'GDPQ1', 'Year', 'GDPQ3', 'TotalPrice', 'TotalP', 'Unemp_rate', 'RNATURALINC', 'RDOMESTICMIG', 'Coast', 'StateCodes', 'TotalE', 'GDPQ2', 'GDP_per_cap', 'Division', 'GDPQ4', 'POPESTIMATE', 'Region', 'RDEATH', 'RNETMIG', 'GDP', 'State', 'RBIRTH']


In [15]:
# Create separate dataframes for each energy type
df_list = [pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()]
for idx, en_list in enumerate(energy_lists):
    df_list[idx] = df_full[list(non_energy + en_list)]


In [16]:
# Drop energy type from column, create new column for energy type
for idx, df_mini in enumerate(df_list):
    df_mini['Energy_type'] = energy_types[idx]
    df_mini.columns = [col.replace(energy_types[idx], '') for col in df_mini.columns]
print(df_list[1].columns)

Index(['Great Lakes', 'RINTERNATIONALMIG', 'FossFuelC', 'TotalC', 'GDPQ1',
       'Year', 'GDPQ3', 'TotalPrice', 'TotalP', 'Unemp_rate', 'RNATURALINC',
       'RDOMESTICMIG', 'Coast', 'StateCodes', 'TotalE', 'GDPQ2', 'GDP_per_cap',
       'Division', 'GDPQ4', 'POPESTIMATE', 'Region', 'RDEATH', 'RNETMIG',
       'GDP', 'State', 'RBIRTH', 'C', 'E', 'P', 'Price', 'Energy_type'],
      dtype='object')


In [17]:
# Recombine the dataFrames, then rename the 'C', 'E', and 'P' columns
import functools
#df = reduce(lambda df1,df2: pd.merge(df1,df2,on='id'), dfList)
df_after = functools.reduce(lambda df1, df2: pd.merge(df1, df2, how='outer'), df_list)
df_after.rename(columns={'C':'Consumption', 'E':'Expenditure', 'P':'Production', 'TotalE':'TotalExpenditures', 'TotalP':'TotalProduction', 'TotalC':'TotalConsumption'}, inplace=True)


In [18]:
# Fill missing price and expenditure values with 0, since energy types that are missing price are hydro, geo, and bio
# These don't cost anything 
df_after[df_after.Price.isnull()]['Energy_type'].unique()
df_after['Price'].fillna(0, inplace=True)
df_after['Expenditure'].fillna(0,inplace=True)

In [19]:
# The missing values for production are for 'Biomass', 'Elec', 'NatGas', 'LPG' which are types that can't be produced
df_after[df_after.Production.isnull()]['Energy_type'].unique()
df_after['Production'].fillna(0, inplace=True)

In [20]:
# Missing migration rates for 2010, will fill those with the average for the column
df_after[df_after.RNETMIG.isnull()]['Year'].unique()
df_after['RNATURALINC'].fillna(df_after['RNATURALINC'].mean(), inplace=True)
df_after['RBIRTH'].fillna(df_after['RBIRTH'].mean(), inplace=True)
df_after['RDOMESTICMIG'].fillna(df_after['RDOMESTICMIG'].mean(), inplace=True)
df_after['RDEATH'].fillna(df_after['RDEATH'].mean(), inplace=True)
df_after['RINTERNATIONALMIG'].fillna(df_after['RINTERNATIONALMIG'].mean(), inplace=True)
df_after['RNETMIG'].fillna(df_after['RNETMIG'].mean(), inplace=True)

In [21]:
# Since United States isn't a state, and its the only one missing values in 'Division', 'Great Lakes', 'Coast', 'Region'
df_after[df_after['Region'].isnull()]['State'].unique()
df_after = df_after[df_after.State != 'United States']


In [22]:
df_after['Unemp_rate'].fillna(df_after['Unemp_rate'].mean(), inplace=True)
df_after.describe()

,Great Lakes,RINTERNATIONALMIG,FossFuelC,TotalConsumption,GDPQ1,Year,GDPQ3,TotalPrice,TotalProduction,Unemp_rate,RNATURALINC,RDOMESTICMIG,Coast,TotalExpenditures,GDPQ2,GDP_per_cap,Division,GDPQ4,POPESTIMATE,Region,RDEATH,RNETMIG,GDP,RBIRTH,Consumption,Expenditure,Production,Price
count,1785.000000,1785.000000,1.785000e+03,1.785000e+03,1.785000e+03,1785.00000,1.785000e+03,1785.000000,1.785000e+03,1785.000000,1785.000000,1785.000000,1785.000000,1785.000000,1.785000e+03,1785.000000,1785.000000,1.785000e+03,1.785000e+03,1785.000000,1785.000000,1785.000000,1.785000e+03,1785.000000,1.785000e+03,1785.000000,1.785000e+03,1785.000000
mean,0.156863,2.450947,1.560458e+06,1.898986e+06,3.088774e+05,2012.00000,3.159242e+05,21.837373,1.477022e+06,7.834320,4.230561,0.403579,0.450980,26408.778431,3.125854e+05,49247.894118,5.117647,3.189102e+05,6.158962e+06,2.666667,8.371682,2.852974,3.140743e+05,12.602243,1.898453e+05,1766.739888,5.938823e+04,9.700566
std,0.363773,1.393915,1.770946e+06,2.036880e+06,3.771367e+05,1.41461,3.865780e+05,3.756488,2.474773e+06,2.166059,2.126430,3.889620,0.497731,28713.098679,3.818960e+05,18424.447793,2.510660,3.917825e+05,6.915771e+06,1.041910,1.072662,3.974580,3.843345e+05,1.367118,3.641683e+05,4208.363645,4.479817e+05,13.325281
min,0.000000,0.557539,4.902500e+04,1.304120e+05,2.571500e+04,2010.00000,2.672900e+04,13.790000,9.500000e+01,3.094713,-0.776457,-13.754494,0.000000,2218.200000,2.632100e+04,31167.000000,1.000000,2.686100e+04,5.643580e+05,1.000000,5.236680,-10.884734,2.640650e+04,9.313705,0.000000e+00,0.000000,0.000000e+00,0.000000
25%,0.000000,1.454160,5.884380e+05,6.400950e+05,7.365700e+04,2011.00000,7.536500e+04,19.390000,2.483700e+05,6.233333,3.056161,-1.484373,0.000000,7884.500000,7.458700e+04,40193.000000,3.000000,7.525000e+04,1.612843e+06,2.000000,7.855200,0.454262,7.443275e+04,12.062215,7.712000e+03,0.000000,0.000000e+00,0.000000
50%,0.000000,2.269191,1.092732e+06,1.428433e+06,1.878460e+05,2012.00000,1.924280e+05,21.330000,6.467380e+05,7.834320,4.234652,0.403579,0.000000,19670.700000,1.898660e+05,45884.000000,5.000000,1.917910e+05,4.383465e+06,3.000000,8.367475,2.856392,1.901705e+05,12.602127,4.166000e+04,142.800000,0.000000e+00,2.600000
75%,0.000000,2.574067,1.761282e+06,2.411816e+06,3.979030e+05,2013.00000,4.059560e+05,23.790000,1.513859e+06,9.077042,5.328165,1.920812,1.000000,33423.700000,3.993660e+05,52099.000000,8.000000,4.131280e+05,6.822112e+06,4.000000,8.970091,4.416764,4.027890e+05,13.100544,2.446460e+05,1498.200000,0.000000e+00,22.390000
max,1.000000,6.883393,1.175266e+07,1.289950e+07,2.274103e+06,2014.00000,2.352770e+06,40.140000,1.759710e+07,13.893276,13.207220,23.189451,1.000000,163261.800000,2.312827e+06,168030.000000,9.000000,2.360282e+06,3.880250e+07,4.000000,11.812924,25.104485,2.324996e+06,18.577841,4.219128e+06,39424.100000,7.658286e+06,99.960000
